# Metadata

**L1 Taxonomy** - Networking

**L2 Taxonomy** - RESTful Services

**Subtopic** - Building a Simple RESTful API with Authentication

**Use Case** - Develop a simple RESTful API in Python using the Flask library. The API should support basic CRUD operations on a resource of your choice. Implement basic authentication using the Flask-HTTPAuth library. All data should be stored and retrieved from a local JSON file.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
Flask==3.1.1
Flask-HTTPAuth==4.8.0
```


# Prompt
## Problem Description  
Develop a simple RESTful API in Python using the Flask framework that manages a collection of Book resources. Each Book has an id, title, author, and year. Implement HTTP Basic Authentication using Flask-HTTPAuth so that only authenticated clients can call any endpoint. Persist all data in a local JSON file so that changes survive server restarts.

## Input Format and Constraints  
- HTTP requests with Content-Type: application/json for request bodies.  
- All endpoints require HTTP Basic Auth credentials (username/password).  
- Book resource schema:
  - id: string (UUID4), generated by the server, unique.
  - title: non-empty string.
  - author: non-empty string.
  - year: integer, four digits.
- JSON file path (e.g. books.json) is provided at application start.
- Use only Flask, Flask-HTTPAuth, and Python standard library.

## Expected Output Format  
- JSON responses with appropriate HTTP status codes:
  - 200 OK for successful GET and PUT.
  - 201 Created for successful POST.
  - 204 No Content for successful DELETE.
  - 400 Bad Request for invalid payloads.
  - 401 Unauthorized for missing/invalid credentials.
  - 404 Not Found for unknown IDs.
  - 500 Internal Server Error for file I/O issues.
- Response bodies:
  - GET /books - list of book objects.
  - GET /books/<id> - single book object.
  - POST /books - created book object (with id).
  - PUT /books/<id> - updated book object.
  - DELETE /books/<id> - empty body.

## Examples

1. List all books  
   GET /books  
   Authorization: Basic <credentials>  
   Response 200 OK  
   [
     {"id":"1","title":"1984","author":"Orwell","year":1949},
     {"id":"2","title":"Dune","author":"Herbert","year":1965}
   ]

2. Get one book  
   GET /books/1  
   Response 200 OK or 404 Not Found

3. Create a book  
   POST /books  
   Body: {"title":"Native Son","author":"Wright","year":1940}  
   Response 201 Created  
   {"id":"3","title":"Native Son","author":"Wright","year":1940}

4. Update a book  
   PUT /books/3  
   Body: {"title":"Native Son Revised","author":"Wright","year":1940}  
   Response 200 OK

5. Delete a book  
   DELETE /books/3  
   Response 204 No Content

6. Authentication failure  
   Any endpoint without valid credentials - 401 Unauthorized.

# Requirements
## Explicit and Implicit Points

1. Endpoints  
   - GET    /books  
   - GET    /books/<id>  
   - POST   /books  
   - PUT    /books/<id>  
   - DELETE /books/<id>

2. Authentication  
   - Use Flask-HTTPAuth for Basic Auth.  
   - All routes require authentication.  
   - Credentials may be hard-coded for simplicity.

3. Persistence  
   - Store books in a local JSON file (books.json).  
   - On startup, load existing data or initialize an empty list.  
   - After any create/update/delete, write the entire list back to disk atomically.

4. Validation  
   - title and author must be non-empty strings.  
   - year must be a four-digit integer.  
   - IDs generated by server (UUID4) must be unique.

5. Error Handling  
   - 400 Bad Request for JSON schema violations.  
   - 401 Unauthorized for missing/invalid credentials.  
   - 404 Not Found for non-existent IDs.  
   - 500 Internal Server Error for file read/write failures.

## Solution Expectations

- A single Flask application file (app.py) defining the routes above.  
- Configuration section for:
  - JSON file path.
  - User credentials.  
- Utility functions to load and save the JSON file.  
- Proper use of Flask-HTTPAuth decorators for authentication.  
- Clean, modular code with one function per concern.

## Function and Class Signatures

```python
def load_books(file_path: str) -> list[dict]:
    # Load the list of books from the JSON file (or return [] if not exists)
    ...
def save_books(file_path: str, books: list[dict]) -> None:
    # Persist the list of books to the JSON file atomically
    ...
def create_app(data_file: str, users: dict[str, str]) -> Flask:
    # Factory to create and configure the Flask app with auth and routes
    ...
```

## Edge Case Behavior

- Empty or missing JSON file - starts with an empty book list.  
- Requests with extra fields - ignore unknown fields.  
- Duplicate title or author allowed; uniqueness only enforced on id.  
- Concurrent writes should not corrupt the file.

## Constraints

- Use only Flask, Flask-HTTPAuth, and Python standard library.  
- Python 3.8+.  
- No additional Flask extensions.

## Important Notes

- Input validation is required; do not assume clients send correct data.  
- Credentials management is simplified; production-grade security not required.  
- Ensure JSON file integrity (e.g., write to a temp file then rename).

In [ ]:
# code
import json
import os
import tempfile
import uuid
from typing import Any, Dict, List, Optional

from flask import Flask, jsonify, request, abort
from flask_httpauth import HTTPBasicAuth


def load_books(file_path: str) -> List[Dict[str, Any]]:
    if not os.path.exists(file_path):
        return []
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except Exception:
        return []
    if isinstance(data, list):
        return [b for b in data if isinstance(b, dict)]
    return []


def save_books(file_path: str, books: List[Dict[str, Any]]) -> None:
    d = os.path.dirname(os.path.abspath(file_path)) or "."
    os.makedirs(d, exist_ok=True)
    fd, tmp_path = tempfile.mkstemp(prefix="books_", suffix=".json", dir=d, text=True)
    try:
        with os.fdopen(fd, "w", encoding="utf-8") as tmp:
            json.dump(books, tmp)
            tmp.flush()
            os.fsync(tmp.fileno())
        os.replace(tmp_path, file_path)
    finally:
        if os.path.exists(tmp_path):
            try:
                os.remove(tmp_path)
            except OSError:
                pass


def _non_empty_str(v: Any) -> str:
    if not isinstance(v, str):
        raise ValueError
    s = v.strip()
    if not s:
        raise ValueError
    return s


def _year_int(v: Any) -> int:
    if not isinstance(v, int):
        raise ValueError
    if v < 1000 or v > 9999:
        raise ValueError
    return v


def _validate_book_payload(payload: Any) -> Dict[str, Any]:
    if not isinstance(payload, dict):
        raise ValueError
    title = _non_empty_str(payload.get("title"))
    author = _non_empty_str(payload.get("author"))
    year = _year_int(payload.get("year"))
    return {"title": title, "author": author, "year": year}


def create_app(data_file: str, users: Dict[str, str]) -> Flask:
    app = Flask(__name__)
    auth = HTTPBasicAuth()

    store: List[Dict[str, Any]] = load_books(data_file)
    app.config["DATA_FILE"] = data_file
    app.config["USERS"] = users
    app.config["ITEM_STORE"] = store

    @auth.verify_password
    def _verify(username: str, password: str) -> Optional[str]:
        if username in users and users[username] == password:
            return username
        return None

    @auth.error_handler
    def _auth_error(_status):
        return jsonify({"error": "Unauthorized"}), 401

    def _parse_json_or_400():
        if not request.is_json:
            abort(400, description="invalid JSON")
        data = request.get_json(silent=True)
        if data is None:
            abort(400, description="invalid JSON")
        return data

    def _persist_or_500():
        try:
            save_books(data_file, store)
        except Exception:
            abort(500, description="file I/O error")

    def _find(book_id: str) -> Optional[Dict[str, Any]]:
        for b in store:
            if b.get("id") == book_id:
                return b
        return None

    @app.route("/books", methods=["GET"])
    @auth.login_required
    def list_books():
        return jsonify(store), 200

    @app.route("/books/<string:book_id>", methods=["GET"])
    @auth.login_required
    def get_book(book_id: str):
        book = _find(book_id)
        if not book:
            abort(404, description="not found")
        return jsonify(book), 200

    @app.route("/books", methods=["POST"])
    @auth.login_required
    def create_book():
        payload = _parse_json_or_400()
        try:
            fields = _validate_book_payload(payload)
        except ValueError:
            abort(400, description="bad request")
        book = {"id": str(uuid.uuid4()), **fields}
        store.append(book)
        _persist_or_500()
        return jsonify(book), 201

    @app.route("/books/<string:book_id>", methods=["PUT"])
    @auth.login_required
    def update_book(book_id: str):
        payload = _parse_json_or_400()
        try:
            fields = _validate_book_payload(payload)
        except ValueError:
            abort(400, description="bad request")
        book = _find(book_id)
        if not book:
            abort(404, description="not found")
        book.update(fields)
        _persist_or_500()
        return jsonify(book), 200

    @app.route("/books/<string:book_id>", methods=["DELETE"])
    @auth.login_required
    def delete_book(book_id: str):
        before = len(store)
        store[:] = [b for b in store if b.get("id") != book_id]
        if len(store) == before:
            abort(404, description="not found")
        _persist_or_500()
        return ("", 204)

    @app.errorhandler(400)
    def _bad_request(err):
        return jsonify({"error": getattr(err, "description", "bad request")}), 400

    @app.errorhandler(404)
    def _not_found(err):
        return jsonify({"error": "not found"}), 404

    @app.errorhandler(500)
    def _server_error(err):
        return jsonify({"error": "internal error"}), 500

    return app


if __name__ == "__main__":
    create_app("books.json", {"admin": "password"}).run(debug=True)


In [ ]:
# tests

"""
Test cases for the Flask Books RESTful API endpoints.

This module contains comprehensive test cases for testing the Flask Books API
endpoints that manage a collection of Book resources with HTTP Basic
Authentication and JSON file persistence.
"""

import unittest
import json
import os
import tempfile
import base64

# Assuming the main application is in app.py
from main import create_app


class TestBooksAPIEndpoints(unittest.TestCase):
    """Test cases for the Books REST API endpoints."""

    def setUp(self):
        """Set up test fixtures before each test method."""
        self.temp_dir = tempfile.mkdtemp()
        self.test_file = os.path.join(self.temp_dir, 'test_books.json')
        self.test_users = {'admin': 'password', 'user': 'secret'}
        self.app = create_app(self.test_file, self.test_users)
        self.client = self.app.test_client()

        # Create valid auth headers
        self.valid_auth = self._create_auth_header('admin', 'password')
        self.invalid_auth = self._create_auth_header('admin', 'wrong')

        # Sample book data
        self.sample_book = {
            'title': 'Test Book',
            'author': 'Test Author',
            'year': 2023
        }

    def tearDown(self):
        """Clean up after each test method."""
        if os.path.exists(self.test_file):
            os.remove(self.test_file)
        os.rmdir(self.temp_dir)

    def _create_auth_header(self, username, password):
        """Create HTTP Basic Auth header."""
        credentials = base64.b64encode(
            f'{username}:{password}'.encode('utf-8')
        ).decode('utf-8')
        return {'Authorization': f'Basic {credentials}'}

    def _create_test_book(self, book_data=None):
        """Helper method to create a test book via POST endpoint."""
        if book_data is None:
            book_data = self.sample_book

        response = self.client.post(
            '/books',
            data=json.dumps(book_data),
            content_type='application/json',
            headers=self.valid_auth
        )
        return response

    def test_get_books_endpoint_no_auth(self):
        """Test GET /books without authentication returns 401."""
        response = self.client.get('/books')
        self.assertEqual(response.status_code, 401)

    def test_get_books_endpoint_invalid_auth(self):
        """Test GET /books with invalid credentials returns 401."""
        response = self.client.get('/books', headers=self.invalid_auth)
        self.assertEqual(response.status_code, 401)

    def test_get_books_endpoint_empty_collection(self):
        """Test GET /books returns empty list when no books exist."""
        response = self.client.get('/books', headers=self.valid_auth)
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, [])
        self.assertEqual(response.content_type, 'application/json')

    def test_get_books_endpoint_with_data(self):
        """Test GET /books returns list of books when data exists."""
        # Create two test books
        book1 = {'title': '1984', 'author': 'Orwell', 'year': 1949}
        book2 = {'title': 'Dune', 'author': 'Herbert', 'year': 1965}

        self._create_test_book(book1)
        self._create_test_book(book2)

        response = self.client.get('/books', headers=self.valid_auth)
        self.assertEqual(response.status_code, 200)
        self.assertEqual(len(response.json), 2)
        self.assertIsInstance(response.json, list)

    def test_get_book_by_id_endpoint_success(self):
        """Test GET /books/<id> returns specific book."""
        # Create a test book first
        create_response = self._create_test_book()
        book_id = create_response.json['id']

        response = self.client.get(f'/books/{book_id}',
                                   headers=self.valid_auth)
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json['title'], self.sample_book['title'])
        self.assertEqual(response.json['id'], book_id)

    def test_get_book_by_id_endpoint_not_found(self):
        """Test GET /books/<id> returns 404 for non-existent book."""
        response = self.client.get('/books/non-existent-id',
                                   headers=self.valid_auth)
        self.assertEqual(response.status_code, 404)

    def test_get_book_by_id_endpoint_no_auth(self):
        """Test GET /books/<id> without authentication returns 401."""
        response = self.client.get('/books/some-id')
        self.assertEqual(response.status_code, 401)

    def test_post_books_endpoint_success(self):
        """Test POST /books creates new book successfully."""
        response = self._create_test_book()

        self.assertEqual(response.status_code, 201)
        self.assertIn('id', response.json)
        self.assertEqual(response.json['title'], self.sample_book['title'])
        self.assertEqual(response.json['author'], self.sample_book['author'])
        self.assertEqual(response.json['year'], self.sample_book['year'])
        self.assertEqual(response.content_type, 'application/json')

    def test_post_books_endpoint_no_auth(self):
        """Test POST /books without authentication returns 401."""
        response = self.client.post(
            '/books',
            data=json.dumps(self.sample_book),
            content_type='application/json'
        )
        self.assertEqual(response.status_code, 401)

    def test_post_books_endpoint_invalid_json(self):
        """Test POST /books with invalid JSON returns 400."""
        response = self.client.post(
            '/books',
            data='invalid json',
            content_type='application/json',
            headers=self.valid_auth
        )
        self.assertEqual(response.status_code, 400)

    def test_post_books_endpoint_missing_title(self):
        """Test POST /books with missing title returns 400."""
        incomplete_book = {'author': 'Test Author', 'year': 2023}
        response = self.client.post(
            '/books',
            data=json.dumps(incomplete_book),
            content_type='application/json',
            headers=self.valid_auth
        )
        self.assertEqual(response.status_code, 400)

    def test_post_books_endpoint_missing_author(self):
        """Test POST /books with missing author returns 400."""
        incomplete_book = {'title': 'Test Book', 'year': 2023}
        response = self.client.post(
            '/books',
            data=json.dumps(incomplete_book),
            content_type='application/json',
            headers=self.valid_auth
        )
        self.assertEqual(response.status_code, 400)

    def test_post_books_endpoint_missing_year(self):
        """Test POST /books with missing year returns 400."""
        incomplete_book = {'title': 'Test Book', 'author': 'Test Author'}
        response = self.client.post(
            '/books',
            data=json.dumps(incomplete_book),
            content_type='application/json',
            headers=self.valid_auth
        )
        self.assertEqual(response.status_code, 400)

    def test_post_books_endpoint_empty_title(self):
        """Test POST /books with empty title returns 400."""
        invalid_book = {
            'title': '',
            'author': 'Test Author',
            'year': 2023
        }
        response = self.client.post(
            '/books',
            data=json.dumps(invalid_book),
            content_type='application/json',
            headers=self.valid_auth
        )
        self.assertEqual(response.status_code, 400)

    def test_post_books_endpoint_empty_author(self):
        """Test POST /books with empty author returns 400."""
        invalid_book = {
            'title': 'Test Book',
            'author': '',
            'year': 2023
        }
        response = self.client.post(
            '/books',
            data=json.dumps(invalid_book),
            content_type='application/json',
            headers=self.valid_auth
        )
        self.assertEqual(response.status_code, 400)

    def test_post_books_endpoint_invalid_year_string(self):
        """Test POST /books with year as string returns 400."""
        invalid_book = {
            'title': 'Test Book',
            'author': 'Test Author',
            'year': 'not a number'
        }
        response = self.client.post(
            '/books',
            data=json.dumps(invalid_book),
            content_type='application/json',
            headers=self.valid_auth
        )
        self.assertEqual(response.status_code, 400)

    def test_post_books_endpoint_year_not_four_digits(self):
        """Test POST /books with year not four digits returns 400."""
        invalid_book = {
            'title': 'Test Book',
            'author': 'Test Author',
            'year': 999
        }
        response = self.client.post(
            '/books',
            data=json.dumps(invalid_book),
            content_type='application/json',
            headers=self.valid_auth
        )
        self.assertEqual(response.status_code, 400)

    def test_post_books_endpoint_year_too_many_digits(self):
        """Test POST /books with year having too many digits returns 400."""
        invalid_book = {
            'title': 'Test Book',
            'author': 'Test Author',
            'year': 12345
        }
        response = self.client.post(
            '/books',
            data=json.dumps(invalid_book),
            content_type='application/json',
            headers=self.valid_auth
        )
        self.assertEqual(response.status_code, 400)

    def test_put_books_endpoint_success(self):
        """Test PUT /books/<id> updates existing book successfully."""
        # Create a book first
        create_response = self._create_test_book()
        book_id = create_response.json['id']

        updated_book = {
            'title': 'Updated Title',
            'author': 'Updated Author',
            'year': 2024
        }

        response = self.client.put(
            f'/books/{book_id}',
            data=json.dumps(updated_book),
            content_type='application/json',
            headers=self.valid_auth
        )

        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json['title'], 'Updated Title')
        self.assertEqual(response.json['author'], 'Updated Author')
        self.assertEqual(response.json['year'], 2024)
        self.assertEqual(response.json['id'], book_id)

    def test_put_books_endpoint_not_found(self):
        """Test PUT /books/<id> returns 404 for non-existent book."""
        response = self.client.put(
            '/books/non-existent-id',
            data=json.dumps(self.sample_book),
            content_type='application/json',
            headers=self.valid_auth
        )
        self.assertEqual(response.status_code, 404)

    def test_put_books_endpoint_no_auth(self):
        """Test PUT /books/<id> without authentication returns 401."""
        response = self.client.put(
            '/books/some-id',
            data=json.dumps(self.sample_book),
            content_type='application/json'
        )
        self.assertEqual(response.status_code, 401)

    def test_put_books_endpoint_invalid_data(self):
        """Test PUT /books/<id> with invalid data returns 400."""
        # Create a book first
        create_response = self._create_test_book()
        book_id = create_response.json['id']

        invalid_update = {
            'title': '',
            'author': 'Test Author',
            'year': 2023
        }

        response = self.client.put(
            f'/books/{book_id}',
            data=json.dumps(invalid_update),
            content_type='application/json',
            headers=self.valid_auth
        )
        self.assertEqual(response.status_code, 400)

    def test_delete_books_endpoint_success(self):
        """Test DELETE /books/<id> removes book successfully."""
        # Create a book first
        create_response = self._create_test_book()
        book_id = create_response.json['id']

        response = self.client.delete(f'/books/{book_id}',
                                      headers=self.valid_auth)
        self.assertEqual(response.status_code, 204)
        self.assertEqual(response.data, b'')

        # Verify book is actually deleted
        get_response = self.client.get(f'/books/{book_id}',
                                       headers=self.valid_auth)
        self.assertEqual(get_response.status_code, 404)

    def test_delete_books_endpoint_not_found(self):
        """Test DELETE /books/<id> returns 404 for non-existent book."""
        response = self.client.delete('/books/non-existent-id',
                                      headers=self.valid_auth)
        self.assertEqual(response.status_code, 404)

    def test_delete_books_endpoint_no_auth(self):
        """Test DELETE /books/<id> without authentication returns 401."""
        response = self.client.delete('/books/some-id')
        self.assertEqual(response.status_code, 401)

    def test_post_books_endpoint_with_extra_fields(self):
        """Test POST /books ignores extra fields in request."""
        book_with_extra = {
            'title': 'Test Book',
            'author': 'Test Author',
            'year': 2023,
            'extra_field': 'should be ignored',
            'another_field': 42
        }

        response = self.client.post(
            '/books',
            data=json.dumps(book_with_extra),
            content_type='application/json',
            headers=self.valid_auth
        )

        self.assertEqual(response.status_code, 201)
        self.assertNotIn('extra_field', response.json)
        self.assertNotIn('another_field', response.json)

    def test_duplicate_books_allowed(self):
        """Test that duplicate book titles and authors are allowed."""
        # Create first book
        response1 = self._create_test_book()
        self.assertEqual(response1.status_code, 201)

        # Create second book with same data
        response2 = self._create_test_book()
        self.assertEqual(response2.status_code, 201)

        # Verify different IDs but same content
        self.assertNotEqual(response1.json['id'], response2.json['id'])
        self.assertEqual(response1.json['title'], response2.json['title'])

        # Verify both books exist in collection
        get_response = self.client.get('/books', headers=self.valid_auth)
        self.assertEqual(len(get_response.json), 2)

    def test_alternative_user_authentication(self):
        """Test authentication with alternative user credentials."""
        alt_auth = self._create_auth_header('user', 'secret')

        # Test with alternative credentials
        response = self.client.get('/books', headers=alt_auth)
        self.assertEqual(response.status_code, 200)

        # Test creating book with alternative credentials
        response = self.client.post(
            '/books',
            data=json.dumps(self.sample_book),
            content_type='application/json',
            headers=alt_auth
        )
        self.assertEqual(response.status_code, 201)

    def test_books_persistence_across_requests(self):
        """Test that books persist across multiple HTTP requests."""
        # Create a book
        create_response = self._create_test_book()
        book_id = create_response.json['id']

        # Verify it exists in a separate request
        get_response = self.client.get(f'/books/{book_id}',
                                       headers=self.valid_auth)
        self.assertEqual(get_response.status_code, 200)
        self.assertEqual(get_response.json['title'],
                         self.sample_book['title'])

        # Create another book
        book2 = {'title': 'Book 2', 'author': 'Author 2', 'year': 2024}
        self._create_test_book(book2)

        # Verify both books exist
        list_response = self.client.get('/books', headers=self.valid_auth)
        self.assertEqual(len(list_response.json), 2)

    def test_endpoint_content_type_validation(self):
        """Test that POST and PUT endpoints handle content type properly."""
        # Test POST with wrong content type
        response = self.client.post(
            '/books',
            data=json.dumps(self.sample_book),
            content_type='text/plain',
            headers=self.valid_auth
        )
        self.assertIn(response.status_code, [400, 415])

        # Test POST with no content type
        response = self.client.post(
            '/books',
            data=json.dumps(self.sample_book),
            headers=self.valid_auth
        )
        self.assertIn(response.status_code, [400, 415])

    def test_endpoint_method_not_allowed(self):
        """Test that unsupported HTTP methods return 405."""
        # Test PATCH method (not supported)
        response = self.client.patch('/books', headers=self.valid_auth)
        self.assertEqual(response.status_code, 405)

        # Test POST on specific book ID (not supported)
        response = self.client.post('/books/some-id',
                                    headers=self.valid_auth)
        self.assertEqual(response.status_code, 405)


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/281787e3-031f-48fb-8340-14940710014c

#### Model code:

```python
import json
import os
import uuid
from flask import Flask, jsonify, request, abort
from flask_httpauth import HTTPBasicAuth

# User credentials (for simplicity, hard-coded)
USERS = {
    "admin": "password"
}

# File path for storing book data
FILE_PATH = "books.json"

# Initialize Flask app and auth
app = Flask(__name__)
auth = HTTPBasicAuth()

# Utility function to load books from a JSON file
def load_books(file_path: str) -> list:
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            try:
                return json.load(f)
            except json.JSONDecodeError:
                return []
    return []

# Utility function to save books to a JSON file
def save_books(file_path: str, books: list) -> None:
    temp_file_path = file_path + ".tmp"
    with open(temp_file_path, "w") as f:
        json.dump(books, f)
    os.rename(temp_file_path, file_path)

# Authentication function
@auth.verify_password
def verify_password(username, password):
    if username in USERS and USERS[username] == password:
        return username

# Load books data
books = load_books(FILE_PATH)

@app.route('/books', methods=['GET'])
@auth.login_required
def get_books():
    return jsonify(books), 200

@app.route('/books/<string:book_id>', methods=['GET'])
@auth.login_required
def get_book(book_id):
    for book in books:
        if book['id'] == book_id:
            return jsonify(book), 200
    abort(404, description="Book not found")

@app.route('/books', methods=['POST'])
@auth.login_required
def create_book():
    data = request.get_json()
    if not data or 'title' not in data or 'author' not in data or 'year' not in data:
        abort(400, description="Bad Request: Missing fields")
    
    # Validate fields
    if not isinstance(data['title'], str) or not data['title']:
        abort(400, description="Invalid 'title'")
    if not isinstance(data['author'], str) or not data['author']:
        abort(400, description="Invalid 'author'")
    if not isinstance(data['year'], int) or not 1000 <= data['year'] <= 9999:
        abort(400, description="Invalid 'year'")

    # Create new book entry
    new_book = {
        'id': str(uuid.uuid4()),
        'title': data['title'],
        'author': data['author'],
        'year': data['year']
    }
    books.append(new_book)
    save_books(FILE_PATH, books)
    return jsonify(new_book), 201

@app.route('/books/<string:book_id>', methods=['PUT'])
@auth.login_required
def update_book(book_id):
    data = request.get_json()
    if not data or 'title' not in data or 'author' not in data or 'year' not in data:
        abort(400, description="Bad Request: Missing fields")
    
    # Validate fields
    if not isinstance(data['title'], str) or not data['title']:
        abort(400, description="Invalid 'title'")
    if not isinstance(data['author'], str) or not data['author']:
        abort(400, description="Invalid 'author'")
    if not isinstance(data['year'], int) or not 1000 <= data['year'] <= 9999:
        abort(400, description="Invalid 'year'")

    # Update book
    for book in books:
        if book['id'] == book_id:
            book.update({
                'title': data['title'],
                'author': data['author'],
                'year': data['year']
            })
            save_books(FILE_PATH, books)
            return jsonify(book), 200

    abort(404, description="Book not found")

@app.route('/books/<string:book_id>', methods=['DELETE'])
@auth.login_required
def delete_book(book_id):
    global books
    new_books = [book for book in books if book['id'] != book_id]
    
    if len(new_books) != len(books):
        books = new_books
        save_books(FILE_PATH, books)
        return '', 204

    abort(404, description="Book not found")

if __name__ == '__main__':
    app.run()
```

### Annotated Issues

- **<Issue>** Missing required create_app factory. All endpoints & config are bound at import time; cannot inject temp path for tests.
- **<Issue>** load_books swallows JSONDecodeError, silently returning [] on corrupt data; spec says 500 Internal Server Error for file I/O issues.